In [1]:
# Initialization cell
try:  # for CS1302 JupyterLite pyodide kernel
    import piplite

    with open("requirements.txt") as f:
        for package in f:
            package = package.strip()
            print("Installing", package)
            await piplite.install(package)
except ModuleNotFoundError:
    pass

import random
import jupytext
import otter
from ipywidgets import interact

grader = otter.Notebook("main.ipynb")
%reload_ext divewidgets

# Pandas

**CS1302 Introduction to Computer Programming**
___

In this lab, we will analyze COVID19 data using a powerful package called [`pandas`](https://pandas.pydata.org/docs/user_guide/index.html).  
The package name comes from *panel data* and *Python for data analysis*.

## Loading CSV Files with Pandas 

[DATA.GOV.HK](https://data.gov.hk/en-data/dataset/hk-dh-chpsebcddr-novel-infectious-agent) provides an [API](https://data.gov.hk/en/help/api-spec#historicalAPI) to retrieve historical data on COVID-19 cases in Hong Kong.

The following uses the `urlencode` function to create the url that links to a csv file containing probable and confirmed cases of COVID-19 by Aug 1st, 2020.

In [2]:
from urllib.parse import urlencode

url_data_gov_hk_get = "https://api.data.gov.hk/v1/historical-archive/get-file"
url_covid_csv = "http://www.chp.gov.hk/files/misc/enhanced_sur_covid_19_eng.csv"
time = "20200801-1204"
url_covid = url_data_gov_hk_get + "?" + urlencode({"url": url_covid_csv, "time": time})

print(url_covid)

https://api.data.gov.hk/v1/historical-archive/get-file?url=http%3A%2F%2Fwww.chp.gov.hk%2Ffiles%2Fmisc%2Fenhanced_sur_covid_19_eng.csv&time=20200801-1204


```{note}

`urlencode` creates a string `'url=<...>&time=<...>'` with some [special symbols encoded](https://www.w3schools.com/tags/ref_urlencode.ASP), e.g.:
- `:` is replaced by `%3A`, and
- `/` is replaced by `%2F`.

```

**Exercise** (simple-encode)

Write a function `simple_encode` that takes in a string and return a string with `:` and `/` encoded as described above.

```{hint}

Use the `replace` method of `str`.

```

In [3]:
def simple_encode(string):
    return string.replace(":", "%3A").replace("/", "%2F")

In [4]:
grader.check("simple-encode")

simple-encode results: All test cases passed!

Like the function `open` that loads a file into memory, `pandas` has a function `read_csv` that loads a csv file. The csv file can even reside on the web:

In [5]:
import pandas as pd

df_covid = pd.read_csv(url_covid)

print(type(df_covid))
df_covid

<class 'pandas.core.frame.DataFrame'>


,Case no.,Report date,Date of onset,Gender,Age,Name of hospital admitted,Hospitalised/Discharged/Deceased,HK/Non-HK resident,Case classification*,Confirmed/probable
0,1,23/01/2020,21/01/2020,M,39,NaN,Discharged,Non-HK resident,Imported case,Confirmed
1,2,23/01/2020,18/01/2020,M,56,NaN,Discharged,HK resident,Imported case,Confirmed
2,3,24/01/2020,20/01/2020,F,62,NaN,Discharged,Non-HK resident,Imported case,Confirmed
3,4,24/01/2020,23/01/2020,F,62,NaN,Discharged,Non-HK resident,Imported case,Confirmed
4,5,24/01/2020,23/01/2020,M,63,NaN,Discharged,Non-HK resident,Imported case,Confirmed
...,...,...,...,...,...,...,...,...,...,...
3268,3269,31/07/2020,26/07/2020,M,22,NaN,To be provided,HK Resident,Local case,Confirmed
3269,3270,31/07/2020,28/07/2020,F,31,NaN,To be provided,HK Resident,Epidemiologically linked with local case,Confirmed
3270,3271,31/07/2020,Asymptomatic,F,36,NaN,To be provided,HK Resident,Epidemiologically linked with local case,Confirmed
3271,3272,31/07/2020,Pending,F,22,NaN,To be provided,HK Resident,Local case,Confirmed


```{note}

The above creates a [`DataFrame` object](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html?highlight=dataframe#pandas.DataFrame):  
- The content of the csv file is displayed as an HTML table conveniently.   
- We can control how much information to show by setting the [display options](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).)

```

**Exercise** (building)

Using the function `pd.read_csv`, load `building_list_eng.csv` as `df_building` from the url `url_building`.  

In [6]:
url_building_csv = "http://www.chp.gov.hk/files/misc/building_list_eng.csv"
time = "20200801-1203"
url_building = (
    url_data_gov_hk_get + "?" + urlencode({"url": url_building_csv, "time": time})
)
df_building = pd.read_csv(url_building)
df_building

,District,Building name,Last date of residence of the case(s),Related probable/confirmed cases
0,Yau Tsim Mong,David Mansion,NaN,1657
1,Yau Tsim Mong,Kimberley Mansion,NaN,1658
2,Wong Tai Sin,Gomme House,NaN,1659
3,North,"Yung Sui House, Yung Shing Court",NaN,1660
4,Sham Shui Po,Shun King Court,NaN,1661
...,...,...,...,...
2160,Sha Tin,MOSTown (Phase 4) (non-residential),29/07/2020,3067
2161,Tuen Mun,San Hui Market (non-residential),29/07/2020,"1471, 1561, 1907,1931, 2154, 2381, 2458, 2598,..."
2162,Sham Shui Po,Shek Kip Mei Estate Commercial Centre (non-res...,29/07/2020,"1813, 1881, 1927, 2067, 2418, 2688, 3038"
2163,Central & Western,Federate Building Fairwood (non-residential),30/07/2020,3158


In [7]:
grader.check("building")

building results: All test cases passed!

## Selecting and Removing columns

We can obtain the column labels of a `Dataframe` using its `columns` attribute.

In [8]:
df_covid.columns

Index(['Case no.', 'Report date', 'Date of onset', 'Gender', 'Age',
       'Name of hospital admitted', 'Hospitalised/Discharged/Deceased',
       'HK/Non-HK resident', 'Case classification*', 'Confirmed/probable'],
      dtype='object')

Using the indexing operator `[]`, a column of a `DataFrame` can be returned as a [`Series` object](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html), which is essentially a named array.   
We can further use the method `value_counts` to return the counts of different values in another `Series` object.

In [9]:
series_gender_counts = df_covid[
    "Gender"
].value_counts()  # return the number of male and female cases

print(type(series_gender_counts))
series_gender_counts

<class 'pandas.core.series.Series'>


F    1648
M    1625
Name: Gender, dtype: int64

**Exercise** (district)

For `df_building`, use the operator `[]` and method `value_counts` to assign `series_district_counts` to a `Series` object that stores the counts of buildings in different district.

In [10]:
series_district_counts = df_building["District"].value_counts()

In [11]:
grader.check("district")

district results: All test cases passed!

In `df_covid`, it appears that the column `Name of hospital admitted` contains no information. We can confirm this by
1. returning the column as a `Series` with `df_covid_cases['Name of hospital admitted']`, and
1. printing an array of unique column values using the method `unique`.

In [12]:
df_covid["Name of hospital admitted"].unique()

array([nan])

**Exercise** (drop)

Drop the column `Name of hospital admitted` from `df_covid` using the `drop` method of the DataFrame. 

```{hint}

Consider reading the documentation of the `drop` method for  
- mutating `df_covid` in place instead of creating a copy of the DataFrame with the column dropped, but
- suppressing error when dropping a column that does not exist or has already been dropped.

```

In [13]:
df_covid.drop(columns="Name of hospital admitted", inplace=True)

In [14]:
grader.check("drop")

drop results: All test cases passed!

## Selecting Rows of DataFrame

We can select the confirmed male cases using the attribute [`loc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) and the indexing operator `[]`.

In [15]:
df_confirmed_male = df_covid.loc[
    (df_covid["Confirmed/probable"] == "Confirmed") & (df_covid["Gender"] == "M")
]
print(type(df_covid.loc))
df_confirmed_male

<class 'pandas.core.indexing._LocIndexer'>


,Case no.,Report date,Date of onset,Gender,Age,Hospitalised/Discharged/Deceased,HK/Non-HK resident,Case classification*,Confirmed/probable
0,1,23/01/2020,21/01/2020,M,39,Discharged,Non-HK resident,Imported case,Confirmed
1,2,23/01/2020,18/01/2020,M,56,Discharged,HK resident,Imported case,Confirmed
4,5,24/01/2020,23/01/2020,M,63,Discharged,Non-HK resident,Imported case,Confirmed
5,6,26/01/2020,21/01/2020,M,47,Discharged,HK resident,Imported case,Confirmed
7,8,26/01/2020,25/01/2020,M,64,Discharged,Non-HK resident,Imported case,Confirmed
...,...,...,...,...,...,...,...,...,...
3264,3265,31/07/2020,25/07/2020,M,54,To be provided,HK Resident,Local case,Confirmed
3265,3266,31/07/2020,30/07/2020,M,69,To be provided,HK Resident,Epidemiologically linked with local case,Confirmed
3267,3268,31/07/2020,23/07/2020,M,61,To be provided,HK Resident,Epidemiologically linked with local case,Confirmed
3268,3269,31/07/2020,26/07/2020,M,22,To be provided,HK Resident,Local case,Confirmed


```{note}

`loc` essentially returns an object that implements the [advanced indexing method](https://numpy.org/doc/stable/user/basics.indexing.html#advanced-indexing) for `__getitem__`. In particular, the above uses [boolean indexing](https://numpy.org/doc/stable/user/basics.indexing.html#boolean-array-indexing).
```

**Exercise** (local)

Assign `df_confirmed_local` to a `DataFrame` of confirmed cases that are local or epidemiologically linked with a local case.

In [16]:
df_confirmed_local = df_covid.loc[
    (df_covid["Confirmed/probable"] == "Confirmed") &
    ((df_covid["Case classification*"] == "Local case") |
        (df_covid["Case classification*"] == "Epidemiologically linked with local case"))
]
df_confirmed_local

,Case no.,Report date,Date of onset,Gender,Age,Hospitalised/Discharged/Deceased,HK/Non-HK resident,Case classification*,Confirmed/probable
15,16,04/02/2020,23/01/2020,F,64,Discharged,HK resident,Local case,Confirmed
16,17,04/02/2020,22/01/2020,M,60,Discharged,HK resident,Local case,Confirmed
17,18,04/02/2020,28/01/2020,M,25,Discharged,HK resident,Local case,Confirmed
18,19,05/02/2020,30/01/2020,F,28,Discharged,HK resident,Epidemiologically linked with local case,Confirmed
19,20,05/02/2020,04/02/2020,F,56,Discharged,HK resident,Epidemiologically linked with local case,Confirmed
...,...,...,...,...,...,...,...,...,...
3268,3269,31/07/2020,26/07/2020,M,22,To be provided,HK Resident,Local case,Confirmed
3269,3270,31/07/2020,28/07/2020,F,31,To be provided,HK Resident,Epidemiologically linked with local case,Confirmed
3270,3271,31/07/2020,Asymptomatic,F,36,To be provided,HK Resident,Epidemiologically linked with local case,Confirmed
3271,3272,31/07/2020,Pending,F,22,To be provided,HK Resident,Local case,Confirmed


In [17]:
grader.check("local")

local results: All test cases passed!

**Exercise** (case-counts)

Write a function `case_counts` that 
- takes an argument `district`, and
- returns the number of cases in `district`. 

```{hint}

Be careful that there can be more than one case for each building and there may be multiple buildings associated with one case. You may want to use the `split` and `strip` methods of `str` to obtain a list of cases from the `Dataframe`.

```

In [18]:
def case_counts(district):
    # print(df_building["District"].value_counts())
    df_district_list = df_building.loc[
        df_building["District"] == district
    ]
    # print(df_district_list)
    # df_cases_set = df_district_list["Related probable/confirmed cases"]
    cases_set = set()
    for cases in df_district_list["Related probable/confirmed cases"]:
        # print(cases.split(","))
        for case in cases.split(","):
            cases_set.add(case.strip())
    return len(cases_set)
# case_counts("Central & Western")

In [19]:
grader.check("case-counts")

case-counts results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [20]:
# extra files to submit
extra_files = []

In [21]:
# Generate the source main.py necessary for grading and similarity check.
jupytext.write(jupytext.read("main.ipynb"), "main.py", fmt="py:percent")

In [22]:
# Generate the zip file to submit.
grader.export(pdf=False, run_tests=False, files=["main.py", *extra_files])